In [ ]:
from google.colab import drive  # Імпортуємо модуль для роботи Google Drive

drive.mount('/content/drive')  # Монтуємо Google Drive до середовища виконання

In [ ]:
# Копіюємо архів з анотаціями вмісту дорожніх знаків
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_annotation.zip /content/
# Копіюємо архів з зображеннями для навчання, частина 0
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_images.train.0.zip /content/
# Копіюємо архів з зображеннями для навчання, частина 1
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_images.train.1.zip /content/
# Копіюємо архів з зображеннями для навчання, частина 2
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_images.train.2.zip /content/
# Копіюємо архів з зображеннями для перевірки
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_images.val.zip /content/
# Копіюємо архів з тестовими зображеннями
!cp /content/drive/MyDrive/GoogleColab/datav2/mtsd_fully_annotated_images.test.zip /content/

In [ ]:
# Імпорт бібліотек
import zipfile
import os

# Функція для розпаковки зображень до хмарних дирекорій
def unzip_file(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)  # Розпаковує всі файли з архіву zip в вказаний каталог
    os.remove(file_path)  # Видаляє zip-файл після розпакування для економії місця на диску

# Створення необхідних каталогів, якщо вони не існують
os.makedirs('/content/mtsd/annotations', exist_ok=True)
os.makedirs('/content/mtsd/images/train', exist_ok=True)
os.makedirs('/content/mtsd/images/val', exist_ok=True)
os.makedirs('/content/mtsd/images/test', exist_ok=True)

# Розпакування анотаційних файлів до каталогу анотацій
unzip_file('/content/mtsd_fully_annotated_annotation.zip', '/content/mtsd/annotations')
# Розпакування зображень для навчання до каталогу train
unzip_file('/content/mtsd_fully_annotated_images.train.0.zip', '/content/mtsd/images/train')
unzip_file('/content/mtsd_fully_annotated_images.train.1.zip', '/content/mtsd/images/train')
unzip_file('/content/mtsd_fully_annotated_images.train.2.zip', '/content/mtsd/images/train')
# Розпакування зображень для перевірки до каталогу val
unzip_file('/content/mtsd_fully_annotated_images.val.zip', '/content/mtsd/images/val')
# Розпакування зображень для тестування до каталогу test
unzip_file('/content/mtsd_fully_annotated_images.test.zip', '/content/mtsd/images/test')

In [ ]:
import shutil

# Функція для переміщення зображень між директоріями
def move_images(src_dir, dst_dir):
    for root, _, files in os.walk(src_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                shutil.move(os.path.join(root, file), dst_dir)  # Переміщує зображення у кінцевий каталог
    shutil.rmtree(src_dir)  # Видаляє вихідний каталог після переміщення для економії місця на диску

# Переміщення зображень для навчання у кінцевий каталог train
move_images('/content/mtsd/images/train/images', '/content/mtsd/images/train')
# Переміщення зображень для перевірки у кінцевий каталог val
move_images('/content/mtsd/images/val/images', '/content/mtsd/images/val')
# Переміщення зображень для тестування у кінцевий каталог test
move_images('/content/mtsd/images/test/images', '/content/mtsd/images/test')

In [ ]:
import json  # Імпортуємо модуль для роботи з JSON
import matplotlib.pyplot as plt  # Імпортуємо пакет для візуалізації даних
from collections import Counter  # Імпортуємо клас Counter з модуля collections для легкого підрахунку елементів

def count_classes(annotations_dir):
    # Ця функція призначена для підрахунку кількості екземплярів кожного класу у наборі даних
    class_counter = Counter()  # Створюємо об'єкт класу Counter для підрахунку
    for root, _, files in os.walk(annotations_dir):  # Проходимо по всіх файлах у директорії з анотаціями
        for file in files:
            if file.endswith('.json'):  # Якщо файл має розширення .json
                file_path = os.path.join(root, file)  # Формуємо повний шлях до файлу
                with open(file_path, 'r') as f:
                    data = json.load(f)  # Завантажуємо дані з файлу JSON
                    for obj in data.get('objects', []):  # Перебираємо всі об'єкти у файлі
                        class_counter[obj['label']] += 1  # Додаємо знайдений клас до лічильника
    return class_counter  # Повертаємо лічильник з підрахованими класами

annotations_dir = '/content/mtsd/annotations/mtsd_v2_fully_annotated/annotations'

# Підрахунок кількості екземплярів кожного класу в анотаціях
class_counts = count_classes(annotations_dir)

N = 20  # Кількість найпопулярніших класів, які ми хочемо відобразити
top_classes = class_counts.most_common(N)[1:]  # Отримуємо N найпопулярніших класів
labels, counts = zip(*top_classes)  # Розділяємо назви класів та кількості екземплярів для побудови графіку

# Побудова стовпчикової діаграми
plt.figure(figsize=(10, 8))  # Встановлюємо розмір графіку
plt.barh(labels, counts)  # Побудова стовпчикової діаграми
plt.xlabel('Number of Instances')  # Підпис осі x
plt.ylabel('Class Label')  # Підпис осі y
plt.title(f'Top {N} Class Distribution in MTSD Annotations')  # Заголовок графіку
plt.gca().invert_yaxis()  # Реверс осі y, щоб найбільш популярні класи були зверху
plt.show()  # Відображаємо графік

In [ ]:
def convert_annotations(json_dir, yolo_dir, classes, file_list):
    os.makedirs(yolo_dir, exist_ok=True)  # Створюємо директорію для зберігання анотацій у форматі YOLO, якщо її ще не існує
    for root, _, files in os.walk(json_dir):  # Проходимо по всіх файлах у директорії з JSON анотаціями
        for file in files:
            if file.endswith('.json') and file.replace('.json', '.jpg') in file_list:  # Якщо файл анотації JSON та відповідний йому файл зображення присутні у списку файлів
                json_path = os.path.join(root, file)  # Формуємо шлях до JSON анотації
                yolo_path = os.path.join(yolo_dir, file.replace('.json', '.txt'))  # Формуємо шлях до вихідного файлу YOLO анотації
                with open(json_path, 'r') as f:  # Відкриваємо JSON анотацію для читання
                    data = json.load(f)  # Завантажуємо дані з JSON анотації
                    yolo_annotations = []  # Створюємо порожній список для зберігання анотацій у форматі YOLO
                    img_width = data['width']  # Отримуємо ширину зображення з анотації
                    img_height = data['height']  # Отримуємо висоту зображення з анотації
                    for obj in data.get('objects', []):  # Перебираємо всі об'єкти у JSON анотації
                        if obj['label'] not in classes:  # Якщо клас об'єкту не входить до списку класів, пропускаємо цей об'єкт
                            continue
                        class_id = classes.index(obj['label'])  # Отримуємо індекс класу об'єкту в списку класів
                        bbox = obj['bbox']  # Отримуємо координати прямокутника, обмежуючого об'єкт
                        x_center = (bbox['xmin'] + bbox['xmax']) / 2 / img_width  # Обчислюємо нормалізовану координату x центра об'єкту
                        y_center = (bbox['ymin'] + bbox['ymax']) / 2 / img_height  # Обчислюємо нормалізовану координату y центра об'єкту
                        width = (bbox['xmax'] - bbox['xmin']) / img_width  # Обчислюємо нормалізовану ширину об'єкту
                        height = (bbox['ymax'] - bbox['ymin']) / img_height  # Обчислюємо нормалізовану висоту об'єкту
                        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")  # Додаємо анотацію у форматі YOLO до списку
                with open(yolo_path, 'w') as yf:  # Відкриваємо файл YOLO анотації для запису
                    yf.write("\n".join(yolo_annotations))  # Записуємо всі анотації у файл у форматі YOLO

def get_image_files(image_dir):
    image_files = []  # Створюємо порожній список для зберігання назв файлів зображень
    for root, _, files in os.walk(image_dir):  # Проходимо по всіх файлах у директорії з зображеннями
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):  # Якщо файл має розширення .jpg, .jpeg або .png
                image_files.append(file)  # Додаємо файл до списку зображень
    return image_files  # Повертаємо список назв файлів зображень

# Отримання списків файлів зображень для навчання та валідації
train_files = get_image_files('/content/mtsd/images/train')  # Отримуємо список файлів зображень для навчання
val_files = get_image_files('/content/mtsd/images/val')  # Отримуємо список файлів зображень для валідації

# Створення директорій для зберігання YOLO анотацій
os.makedirs('/content/mtsd/labels/train', exist_ok=True)  # Створюємо директорію для навчальних анотацій YOLO
os.makedirs('/content/mtsd/labels/val', exist_ok=True)  # Створюємо директорію для валідаційних анотацій YOLO

# Конвертація анотацій у форматі JSON в формат YOLO
convert_annotations('/content/mtsd/annotations/mtsd_v2_fully_annotated/annotations', '/content/mtsd/labels/train', list(class_counts.keys()), train_files)
convert_annotations('/content/mtsd/annotations/mtsd_v2_fully_annotated/annotations', '/content/mtsd/labels/val', list(class_counts.keys()), val_files)

In [ ]:
# Формування YAML, що містить інформацію про шляхи до даних, кількість класів та їх назви
data_yaml = f"""
train: /content/mtsd/images/train
val: /content/mtsd/images/val
test: /content/mtsd/images/test

nc: {len(class_counts)}
names: {list(class_counts.keys())}
"""

with open('/content/mtsd/data.yaml', 'w') as f:  # Відкриття файлу для запису
    f.write(data_yaml)  # Запис рядка YAML у файл

In [ ]:
import matplotlib.pyplot as plt  # Імпорт модуля для візуалізації даних
import matplotlib.patches as patches  # Імпорт модуля для роботи з графічними об'єктами
import cv2  # Імпорт бібліотеки OpenCV для роботи з зображеннями

def visualize_annotations(image_path, annotation_path, class_names):
    image = cv2.imread(image_path)  # Завантаження зображення за вказаним шляхом
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Конвертація зображення з формату BGR до RGB
    fig, ax = plt.subplots(1)  # Створення нового графічного вікна та осей

    ax.imshow(image)  # Відображення зображення на графіку

    with open(annotation_path, 'r') as f:  # Відкриття файлу анотації для читання
        lines = f.readlines()  # Читання всіх рядків анотації
        for line in lines:  # Перебір кожного рядка анотації
            class_id, x_center, y_center, width, height = map(float, line.split())  # Розбивка рядка на окремі значення
            x_center, y_center, width, height = int(x_center * image.shape[1]), int(y_center * image.shape[0]), int(width * image.shape[1]), int(height * image.shape[0])  # Конвертація нормалізованих координат у пікселі
            x_min, y_min = x_center - width // 2, y_center - height // 2  # Обчислення координат верхнього лівого кута прямокутника
            rect = patches.Rectangle((x_min, y_min), width, height, linewidth=1, edgecolor='g', facecolor='none')  # Створення прямокутника для відображення області анотації
            ax.add_patch(rect)  # Додавання прямокутника до відображення
            ax.text(x_min, y_min - 5, class_names[int(class_id)], fontsize=8, color='g')  # Додавання тексту з назвою класу над прямокутником

    plt.show()  # Відображення зображення з анотаціями

# Шлях до зображення та анотації, а також список назв класів
image_path = '/content/mtsd/images/train/0ghzRywyo186i5Q_Soi_pg.jpg'
annotation_path = '/content/mtsd/labels/train/0ghzRywyo186i5Q_Soi_pg.txt'
visualize_annotations(image_path, annotation_path, list(class_counts.keys()))  # Виклик функції візуалізації

In [ ]:
# Клонування репозиторію YOLOv5 з GitHub
!git clone https://github.com/ultralytics/yolov5
# Перехід до каталогу з клонованим репозиторієм
%cd yolov5
# Встановлення всіх необхідних залежностей за допомогою файлу requirements.txt
!pip install -r requirements.txt

In [ ]:
# Команда для тренування моделі YOLOv5
!python train.py \
    --img 1280 \  # Розмір зображення, що використовується для тренування
    --batch 16 \  # Розмір партії (кількість зображень, що обробляється одночасно)
    --epochs 10 \  # Кількість епох тренування
    --data "/content/mtsd/data.yaml" \  # Шлях до файлу конфігурації даних YAML
    --weights yolov5m6.pt \  # Ваги попередньо навченої моделі, які будуть використані для початкового тренування
    --cache  # Використовувати кеш для швидкого завантаження даних під час тренування

In [ ]:
# optional
!python train.py --img 1280 --batch 16 --epochs 20 --data "/content/mtsd/data.yaml" --weights runs/train/exp/weights/last.pt --cache

In [ ]:
import os  # Імпортуємо модуль для роботи з операційною системою
import shutil  # Імпортуємо модуль для роботи з файловою системою

source_dir = '/content/yolov5'  # Вказуємо директорію, яку ми хочемо скопіювати
destination_dir = '/content/drive/MyDrive/GoogleColab/yolov5_project'  # Вказуємо цільову директорію, куди ми хочемо скопіювати

# Перевіряємо, чи існує цільова директорія, і якщо так, видаляємо її
if os.path.exists(destination_dir):
    shutil.rmtree(destination_dir)

# Копіюємо вміст джерелої директорії у цільову директорію
shutil.copytree(source_dir, destination_dir)